In [1]:
import pandas as pd   
import numpy as np

from bs4 import BeautifulSoup 
import re
from nltk.corpus import stopwords # Import the stop word list
from nltk.stem.porter import PorterStemmer

def clean_review(raw_review):
	example1 = BeautifulSoup(raw_review, 'html.parser').get_text()
	# print type(example1)
	# Use regular expressions to do a find-and-replace
	letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
	                      " ",                   # The pattern to replace it with
	                      example1 )  # The text to search
	# print letters_only
	lower_case = letters_only.lower()        # Convert to lower case
	words = lower_case.split()               # Split into words
	# print words
	# print stopwords.words("english")
	words = [w for w in words if not w in stopwords.words("english")]
	# print words
	from nltk.stem.porter import PorterStemmer
	porter_stemmer = PorterStemmer()
	stemmed_words = []
	for i in range(len(words)):
		stemmed_words.append(porter_stemmer.stem(words[i]))
	return( " ".join( stemmed_words ))


train = pd.read_csv("data\IMDB\labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
print(train.shape)
# print train.columns.values
num_reviews = train["review"].size
clean_reviews = []
for i in range(10):
	if( (i+1)%1000 == 0 ):
		print("Review %d of %d\n" % ( i+1, num_reviews )   )                                                       
	clean_reviews.append(clean_review(train["review"][i]))
%store clean_reviews

(25000, 3)


Stored 'clean_reviews' (list)


In [2]:
print(len(clean_reviews))


for i in range(len(clean_reviews)):
    clean_reviews[i] = clean_reviews[i].split(' ')


print(clean_reviews[1])

10
['classic', 'war', 'world', 'timothi', 'hine', 'entertain', 'film', 'obvious', 'goe', 'great', 'effort', 'length', 'faith', 'recreat', 'h', 'g', 'well', 'classic', 'book', 'mr', 'hine', 'succe', 'watch', 'film', 'appreci', 'fact', 'standard', 'predict', 'hollywood', 'fare', 'come', 'everi', 'year', 'e', 'g', 'spielberg', 'version', 'tom', 'cruis', 'slightest', 'resembl', 'book', 'obvious', 'everyon', 'look', 'differ', 'thing', 'movi', 'envis', 'amateur', 'critic', 'look', 'critic', 'everyth', 'other', 'rate', 'movi', 'import', 'base', 'like', 'entertain', 'peopl', 'never', 'agre', 'critic', 'enjoy', 'effort', 'mr', 'hine', 'put', 'faith', 'h', 'g', 'well', 'classic', 'novel', 'found', 'entertain', 'made', 'easi', 'overlook', 'critic', 'perceiv', 'shortcom']


In [3]:
clean_test_reviews = [] 


for i in range(10,15):
    
    clean_test_reviews.append( clean_review( train["review"][i] ))
    
test_op = train["sentiment"][10:15]
print(test_op)

10    0
11    1
12    1
13    0
14    0
Name: sentiment, dtype: int64


In [11]:
from lib.tensorflowglove import tf_glove
glovemodel = tf_glove.GloVeModel(embedding_size=300, context_size=10)
glovemodel.fit_to_corpus(clean_reviews)

In [12]:
glovemodel.train(num_epochs=10)

In [13]:
print(type(test_op))
print(clean_test_reviews[5])

<class 'pandas.core.series.Series'>


IndexError: list index out of range

In [14]:
import matplotlib
%matplotlib inline
glovemodel.generate_tsne("sample_tsne.png")

In [15]:
glovemodel.embeddings

array([[ 0.10530826,  0.02363008,  1.59077251, ..., -0.63448429,
        -0.86648047,  0.47548783],
       [-1.17545474,  0.04726669, -0.57567799, ..., -0.6307568 ,
         1.32035685, -1.55405879],
       [ 1.24755621,  0.45703289, -1.70392239, ..., -0.41174126,
        -0.57128203,  0.63591558],
       ..., 
       [ 0.0372749 ,  0.14129621, -1.49726582, ...,  0.87931609,
        -0.34475458, -1.074898  ],
       [-1.88765693, -1.15654194,  0.58350301, ...,  0.39413834,
        -0.2064212 ,  0.12882107],
       [-0.35581797, -0.21454969,  1.72506869, ..., -0.27080706,
         0.14710802,  0.62318963]], dtype=float32)

In [18]:
#glovemodel.embeddings[glovemodel.id_for_word('kilo')]

In [19]:
print(glovemodel.max_vocab_size)

100000


In [20]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

Using TensorFlow backend.


In [21]:
max_features = 20000
maxlen = 150  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

In [22]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(clean_reviews, maxlen=maxlen)
x_test = sequence.pad_sequences(clean_test_reviews, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)


ValueError: invalid literal for int() with base 10: 'call'

In [23]:
model = Sequential()
model.add(Embedding(glovemodel.max_vocab_size, 300))
model.add(LSTM(300, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs

In [24]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [25]:
print('Train...')
model.fit(x_train, np.array(train["sentiment"][0:10]),
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test,np.array(test_op)))

Train...


NameError: name 'x_train' is not defined

In [26]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

NameError: name 'x_test' is not defined

In [27]:
print(np.array(train["sentiment"][0:10])[0])
print((clean_reviews[1]))

1
['classic', 'war', 'world', 'timothi', 'hine', 'entertain', 'film', 'obvious', 'goe', 'great', 'effort', 'length', 'faith', 'recreat', 'h', 'g', 'well', 'classic', 'book', 'mr', 'hine', 'succe', 'watch', 'film', 'appreci', 'fact', 'standard', 'predict', 'hollywood', 'fare', 'come', 'everi', 'year', 'e', 'g', 'spielberg', 'version', 'tom', 'cruis', 'slightest', 'resembl', 'book', 'obvious', 'everyon', 'look', 'differ', 'thing', 'movi', 'envis', 'amateur', 'critic', 'look', 'critic', 'everyth', 'other', 'rate', 'movi', 'import', 'base', 'like', 'entertain', 'peopl', 'never', 'agre', 'critic', 'enjoy', 'effort', 'mr', 'hine', 'put', 'faith', 'h', 'g', 'well', 'classic', 'novel', 'found', 'entertain', 'made', 'easi', 'overlook', 'critic', 'perceiv', 'shortcom']
